<h1><center>Context Managers and Else blocks</center></h1>

#### Book: Fluent Python https://www.amazon.co.uk/Fluent-Python-Luciano-Ramalho/dp/1491946008
**Chapter:** Part V. Control flow - Chapter 15. Context Managers and else blocks

The subjects of this chapter are control flow features that are commonly overlooked or underused features in Python  
- The **with** statement and context managers
- the **else** clause in **for**, **while** and **try** statements

## Else blocks beyond if

The meaning of for/else, while/else and try/else are closely related, but very different from if/else

### For/else

- The <em>else</em> block will run only if and when the for loop runs to completion, i.e. not if the for is aborted with a break

In [ ]:
for i in range(3):
    print(i)
else:
    print("else block executed")
    
print("outside of the for/else")

In [ ]:
for i in range(3):
    print(i)
    if i == 1:
        print("breaking out of the loop")
        break
else:
    print("else block executed")
    
print("outside of the for/else")

### While/else

- the <em>else</em> block will run only if and when the while loop exits because the condition became <em>falsy</em>; i.e. not when the while is aborted with a break

In [ ]:
x = 0
while x <= 3:
    print(x)
    x += 1
else:
    print("else block executed")

print("outside of the while/else")

In [ ]:
x = 0
while x <= 3:
    print(x)
    if x == 2:
        print("breaking out of the loop")
        break
    x += 1
else:
    print("else block executed")
    
print("outside of the while/else")

### Try/else

- The <em>else</em> block will only run if no exception is raised in the try block. Also, the official docs say "Exceptions in the else clause are not handled by the preciding <em>except</em> clauses"

In [ ]:
try:
    print("running dangerous code")
except ValueError:
    print("Exception handled")
else:
    print("else block executed")
    
print("outside of the try/else")

In [ ]:
try:
    print("running dangerous code")
    raise ValueError("something bad happened...")
except ValueError:
    print("ValueError exception handled")
else:
    print("else block executed")
    
print("outside of the try/else")

In Python, ***try/except*** is commonly used for control flow, not just for error handling
- There are  two famous acronyms/slogans that originated from this:  
1) EAFP -> "Easier to ask for forgiveness than permission"  
2) LBYL -> "Look before you leap"

In [ ]:
def dangerous_call(num):
    return 2/num

def after_call():
    print("all good")
    
try:
    dangerous_call(1)
    after_call() # this code should be in else block
except ZeroDivisionError:
    print("ZeroDivisionError exception handled")
else:
    print("else block executed")
    
print("outside of the try/else")

The better way of controling our flow is to execute the after call in the else method, because it would only ever be executed if the dangereous code call succeeds. 

And now it's clear that the try block is ***guarding*** against possible errors in ***dangerous_call()*** and not in ***after_call()***. It is also more obvious that ***after_call()*** will only execute if no exception is raised in the try block

In [ ]:
try:
    dangerous_call(1)
except ZeroDivisionError:
    print("ZeroDivisionError exception handled")
else: # else is run only if the code in the try block succeeds
    after_call()
    print("else block executed")
    
print("outside of the try/else")

## Context Managers

Context manager objects exist to control a <em>with</em> statement, just like iterators exist to control a <em>for</em> statement

The ***with*** statement was designed to simplify ***try/finally*** pattern which guarantees that some operation is performed after a block of code, even when exception is raised, a ***return*** or ***sys.exit()*** call. The code in ***finally*** clause usually releases critical resource or restores some previous state that was temporarily changed.
    

In [ ]:
with open('my_file.txt', 'w') as opened_file:
    opened_file.write('Hola!')
    my_var = 1

print(my_var)
print(opened_file)
opened_file.closed

In [ ]:
file = open('my_file2.txt', 'w')
try:
    file.write('Hola!')
finally:
    file.close()

### Class based Context Manager

In [ ]:
class FileHandler():
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, ex_type, value, traceback):
        self.file_obj.close()

While comparing it to the first example a lot of boilerplate code is eliminated just by using ***with***. The main advantage of using a with statement is that it makes sure our file is closed without paying attention to how the nested block exits.

In [ ]:
with FileHandler('my_file3.txt', 'w') as opened_file:
    opened_file.write('Hola!')

## Exception Handling

In [ ]:
with FileHandler('my_file4.txt', 'w') as opened_file:
    opened_file.write('Hola!')
    opened_file.undefined_function('Oops')

In [ ]:
class FileHandler():
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, ex_type, value, traceback):
        self.file_obj.close()
        return False
    
with FileHandler('my_file6.txt', 'w') as opened_file:
    opened_file.write('Hola!')
    opened_file.undefined_function('Oops')

In [ ]:
class FileHandler(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, ex_type, value, traceback):
        self.file_obj.close()
        return True
    
with FileHandler('my_file5.txt', 'w') as opened_file:
    opened_file.write('Hola!')
    opened_file.undefined_function('Oops')

### Example control flow and exception handling

In [ ]:
import os 

class FileHandler(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, ex_type, value, traceback):
        self.file_obj.close()
        if ex_type:
            print(f"Logging Exception type: {ex_type}, value: {value}")
            print(f"Removing corrupted file: {self.file_obj.name}")
            os.remove(self.file_obj.name)
            return True
        
    
with FileHandler('my_file10.txt', 'w') as opened_file:
    opened_file.write('Hola!')
    opened_file.undefined_function('Oops')
    
# is run only if no exception raised or when exception is silenced by the context manager
print("I want to run this regardless of context manager failing")

Silencing/suppressing exceptions in the context manager can be useful when working with AWS Lambda functions as any raised exception automatically stops the lambda function

### Handling specific exception type 

In [ ]:
class FileHandler(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, ex_type, value, traceback):
        self.file_obj.close()
        if ex_type:
            print(f"Logging Exception type: {ex_type}, value: {value}")
            if ex_type == AttributeError:
                print(f"Removing corrupted file: {self.file_obj.name}")
                os.remove(self.file_obj.name)
                return True
            print(f"Logging and raising different Exception type: {ex_type}, value: {value}")
            return False
        
with FileHandler('my_file10.txt', 'w') as opened_file:
    opened_file.write('Hola!')
    opened_file.undefined_function('Oops')
    #raise Exception("Fail")
    
# is run only if no exception raised or when exception is silenced by the context manager
print("I want to run this regardless of context manager failing")

### Context manager as a Generator

Instead of a class, we can implement a Context Manager using a generator function with ***contextlib*** module

In [ ]:
from contextlib import contextmanager

@contextmanager
def open_file(name):
    f = open(name, 'w')
    try:
        yield f
    finally:
        f.close()

1) Python encounters the ***yield*** keyword. Due to this it creates a generator instead of a normal function.  
2) Due to the ***decoration***, contextmanager is called with the function name (open_file) as its argument.  
3) The context manager ***decorator*** returns the generator wrapped by the ***GeneratorContextManager*** object.  
4) The GeneratorContextManager is assigned to the open_file function. Therefore, when we later call the open_file function, we are actually calling the GeneratorContextManager object.

In [ ]:
with open_file('last_file.txt') as f:
    f.write('last hola!')